## Completions

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/completions)


In [ ]:
# Completion
import openai

openai.api_base = "http://localhost:8091/v1"
openai.api_key = ""
prompt = "Tell me something funny about llamas."

response = openai.Completion.create(
    model="llamacpp",
    prompt=prompt,
    temperature=1.31,
    max_tokens=8192,
    top_p=1.0,
    frequency_penalty=0,
    presence_penalty=0,
    stream=False,
)
message = response.choices[0].text.strip()
print(message)

## Chat Completion

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/chat)


In [ ]:
import openai

openai.api_base = "http://localhost:8091/v1"
openai.api_key = ""
prompt = "Tell me something funny about llamas."
messages = [{"role": "system", "content": prompt}]

response = openai.ChatCompletion.create(
    model="Local-LLM",
    messages=messages,
    temperature=1.31,
    max_tokens=8192,
    top_p=1.0,
    n=1,
    stream=False,
)
message = response.choices[0].message.content.strip()
print(message)


## Embeddings

[OpenAI API Reference](https://platform.openai.com/docs/api-reference/embeddings)

The embeddings endpoint it currently uses is an ONNX embedder with 256 max tokens.


In [20]:
import openai

openai.api_base = "http://localhost:8091/v1"
openai.api_key = ""
prompt = "Tell me something funny about llamas."

response = openai.Embedding.create(
    input=prompt,
    model="Local-LLM",
)

print(response)

{
  "data": [
    {
      "embedding": [
        [
          -0.020691318437457085,
          0.05344342440366745,
          -0.01896013133227825,
          0.06367390602827072,
          -0.02409016527235508,
          -0.06056477501988411,
          0.018430354073643684,
          -0.08217457681894302,
          -0.010573913343250751,
          0.07694689929485321,
          4.143632031627931e-05,
          -0.06921520829200745,
          0.05988619849085808,
          -0.0015787760494276881,
          -0.041967082768678665,
          0.002724555553868413,
          0.06972332298755646,
          -0.0032384481746703386,
          0.004325220361351967,
          0.06860488653182983,
          0.03619697317481041,
          0.04522477090358734,
          0.05397801846265793,
          0.0010990725131705403,
          -0.058630287647247314,
          0.04101470112800598,
          -0.01772269979119301,
          0.008305024355649948,
          -0.050267696380615234,
          -0.0087099